In [55]:
import pandas as pd

- `results.csv` joins `races.csv` to get `year`
- `results.csv` joins `drivers.csv` to get `driver`
- `results.csv` joins `constructors.csv` to get `constructor`
- `results.csv` joins `pitStops.csv` to get `duration`

In [56]:
results = pd.read_csv('source/results.csv')
results

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,25841,1096,854,210,47,12,16,16,16,0.0,57,\N,\N,39,12,1:29.833,211.632,11
25836,25842,1096,825,210,20,16,17,17,17,0.0,57,\N,\N,40,20,1:31.158,208.556,11
25837,25843,1096,1,131,44,5,18,18,18,0.0,55,\N,\N,42,11,1:29.788,211.738,9
25838,25844,1096,849,3,6,20,19,19,19,0.0,55,\N,\N,45,14,1:30.309,210.517,130


In [57]:
results = pd.read_csv('source/results.csv')
results = results[['raceId', 'driverId', 'constructorId', 'points', 'position']]

races = pd.read_csv('source/races.csv')
races.rename(columns={'name': 'raceName'}, inplace=True)
races = races[['raceId', 'year', 'date', 'raceName']]

drivers = pd.read_csv('source/drivers.csv')
drivers['driverFullname'] = drivers['forename'] + ' ' + drivers['surname']
drivers = drivers[['driverId', 'driverFullname']]
    
constructors = pd.read_csv('source/constructors.csv')
constructors.rename(columns={'name': 'constructorName'}, inplace=True)
constructors = constructors[['constructorId', 'constructorName']]

pitstops = pd.read_csv('source/pit_stops.csv')
pitstops = pitstops[['raceId', 'driverId', 'stop', 'duration', 'milliseconds']]

In [58]:
print(results.shape)
results = results.merge(races, on='raceId', how='inner')
print(results.shape)
results = results.merge(drivers, on='driverId', how='left')
print(results.shape)
results = results.merge(constructors, on='constructorId', how='left')
print(results.shape)
results = results.merge(pitstops, on=['raceId', 'driverId'], how='left')
print(results.shape)

(25840, 5)
(25840, 8)
(25840, 9)
(25840, 10)
(30773, 13)


---

### Getting `times.csv`

In [59]:
columnsTimes = ['year', 'raceName', 'driverFullname', 'constructorName', 'position', 'stop', 'milliseconds']
times = results[columnsTimes].sort_values(by=['year']) \
                             .dropna(subset=['milliseconds'])

times['duration'] = times['milliseconds'] / 1000

times.to_csv('wrang/times.csv', index=False)
times.to_csv('../../Public-Viz-F1/times.csv', index=False)

---
### Getting `driversWinners.csv` & `constructorsWinners.csv`

In [60]:
columnsDriverWinners = ['year', 'raceId', 'date', 'raceName', 'driverFullname', 'points']
totalPoints = results[columnsDriverWinners].drop_duplicates() \
              .groupby(['year', 'driverFullname'])['points'] \
              .sum() \
              .reset_index(name='points')
              
# keep only the winners per year
driversWinners = totalPoints.loc[totalPoints.groupby('year')['points'].idxmax()] 

# change the winner in 1964
driversWinners.loc[driversWinners['year'] == 1964, 'driverFullname'] = 'John Surtees'

# change the winner in 1988
driversWinners.loc[driversWinners['year'] == 1988, 'driverFullname'] = 'Ayrton Senna'

# indicator variable for the winners
driversWinners['winner'] = 1

# get the cumulative count of wins
driversWinners['wins'] = driversWinners.groupby(['driverFullname'])['winner'].cumsum()

# Get unique car drivers
car_drivers = driversWinners['driverFullname'].unique()

# Create a new dataframe with all years and cumulative sum for each driver
years = range(driversWinners['year'].min(), driversWinners['year'].max() + 1)
newDriversWinners = pd.DataFrame({'year': years})
for driver in car_drivers:
    drivers_cumsum = driversWinners[driversWinners['driverFullname'] == driver]['year'].value_counts().sort_index().cumsum()
    newDriversWinners[driver] = newDriversWinners['year'].map(drivers_cumsum).fillna(method='ffill').fillna(0)
    # method = 'ffill' gets the last non-null value and fills the nulls with it

# pivot to make the columns turn into rows
newDriversWinners = newDriversWinners.melt(id_vars=['year'], var_name='driverFullname', value_name='wins')

newDriversWinners.to_csv('wrang/driversWinners.csv', index=False)
newDriversWinners.to_csv('../../Public-Viz-F1/driversWinners.csv', index=False)

newDriversWinners

,year,driverFullname,wins
0,1950,Nino Farina,1.0
1,1951,Nino Farina,1.0
2,1952,Nino Farina,1.0
3,1953,Nino Farina,1.0
4,1954,Nino Farina,1.0
...,...,...,...
2477,2018,Max Verstappen,0.0
2478,2019,Max Verstappen,0.0
2479,2020,Max Verstappen,0.0
2480,2021,Max Verstappen,1.0


In [61]:
columnsConstructorsWinners = ['year', 'raceId', 'date', 'raceName', 'constructorName', 'points']

flt = results['year'] >= 1958

totalPoints = results[columnsConstructorsWinners][flt].drop_duplicates() \
              .groupby(['year', 'constructorName'])['points'] \
              .sum() \
              .reset_index(name='points')
              
# keep only the winners per year
constructorsWinners = totalPoints.loc[totalPoints.groupby('year')['points'].idxmax()] 

# change the winner in 1964
constructorsWinners.loc[constructorsWinners['year'] == 1964, 'constructorName'] = 'Ferrari'

# change the winner in 1988
constructorsWinners.loc[constructorsWinners['year'] == 1988, 'constructorName'] = 'McLaren'

# indicator variable for the winners
constructorsWinners['winner'] = 1

# get the cumulative count of wins
constructorsWinners['wins'] = constructorsWinners.groupby(['constructorName'])['winner'].cumsum()

# Get unique car constructors
car_constructors = constructorsWinners['constructorName'].unique()

# Create a new dataframe with all years and cumulative sum for each constructor
years = range(constructorsWinners['year'].min(), constructorsWinners['year'].max() + 1)
newConstructorsWinners = pd.DataFrame({'year': years})
for constructor in car_constructors:
    constructor_cumsum = constructorsWinners[constructorsWinners['constructorName'] == constructor]['year'].value_counts().sort_index().cumsum()
    newConstructorsWinners[constructor] = newConstructorsWinners['year'].map(constructor_cumsum).fillna(method='ffill').fillna(0)
    # method = 'ffill' gets the last non-null value and fills the nulls with it

# pivot to make the columns turn into rows
newConstructorsWinners = newConstructorsWinners.melt(id_vars=['year'], var_name='constructorName', value_name='wins')

newConstructorsWinners.to_csv('wrang/constructorsWinners.csv', index=False)
newConstructorsWinners.to_csv('../../Public-Viz-F1/constructorsWinners.csv', index=False)

In [62]:
columnsConstructorsWinnersByRace = ['year', 'raceId', 'date', 'raceName', 'constructorName', 'position']
totalWins = results[columnsConstructorsWinnersByRace].query("position == '1'").drop_duplicates() \
              .groupby(['year', 'constructorName'])['position'] \
              .count() \
              .reset_index(name='wins')

# get the cumulative count of wins
totalWins['wins_cum'] = totalWins.groupby(['constructorName'])['wins'].cumsum()

totalWins = totalWins.pivot(index='year', columns='constructorName', values='wins_cum').fillna(method='ffill').fillna(0).reset_index()
newConstructorsWinnersByRace = totalWins.melt(id_vars=['year'], var_name='constructorName', value_name='wins_cum')

newConstructorsWinnersByRace.to_csv('wrang/constructorRaceWins.csv', index=False)
newConstructorsWinnersByRace.to_csv('../../Public-Viz-F1/constructorRaceWins.csv', index=False)

newConstructorsWinnersByRace

,year,constructorName,wins_cum
0,1950,Alfa Romeo,6.0
1,1951,Alfa Romeo,10.0
2,1952,Alfa Romeo,10.0
3,1953,Alfa Romeo,10.0
4,1954,Alfa Romeo,10.0
...,...,...,...
3426,2018,Wolf,3.0
3427,2019,Wolf,3.0
3428,2020,Wolf,3.0
3429,2021,Wolf,3.0


---

### Getting `positions.csv`

In [63]:
positionsColumns = ['year', 'date', 'raceName', 'driverFullname', 'points', 'position']
positions = results[positionsColumns].drop_duplicates() 

positions.to_csv('wrang/positions.csv', index=False)
positions

,year,date,raceName,driverFullname,points,position
0,2008,2008-03-16,Australian Grand Prix,Lewis Hamilton,10.0,1
1,2008,2008-03-16,Australian Grand Prix,Nick Heidfeld,8.0,2
2,2008,2008-03-16,Australian Grand Prix,Nico Rosberg,6.0,3
3,2008,2008-03-16,Australian Grand Prix,Fernando Alonso,5.0,4
4,2008,2008-03-16,Australian Grand Prix,Heikki Kovalainen,4.0,5
...,...,...,...,...,...,...
30766,2022,2022-11-20,Abu Dhabi Grand Prix,Mick Schumacher,0.0,16
30768,2022,2022-11-20,Abu Dhabi Grand Prix,Kevin Magnussen,0.0,17
30769,2022,2022-11-20,Abu Dhabi Grand Prix,Lewis Hamilton,0.0,18
30770,2022,2022-11-20,Abu Dhabi Grand Prix,Nicholas Latifi,0.0,19


In [22]:
#json listing by year the drivers and constructors

participationColumns = ['year',  'driverFullname']
participation = results[participationColumns].drop_duplicates()

#group by year and convert to a dict with yaer: [list of drivers]
participation = participation.groupby('year')['driverFullname'].apply(list).reset_index(name='drivers')

#convert to a dict with yaer: [list of drivers]
participation = dict(zip(participation['year'], participation['drivers']))

#convert to a json
import json
with open('wrang/participation.json', 'w') as fp:
    json.dump(participation, fp)
    

participation

{1950: ['Nino Farina',
  'Luigi Fagioli',
  'Reg Parnell',
  'Yves Cabantous',
  'Louis Rosier',
  'Bob Gerard',
  'Cuth Harrison',
  'Philippe Étancelin',
  'David Hampshire',
  'Brian Shawe Taylor',
  'Johnny Claes',
  'Juan Fangio',
  'Joe Kelly',
  'Prince Bira',
  'David Murray',
  'Geoff Crossley',
  'Toulo de Graffenried',
  'Louis Chiron',
  'Eugène Martin',
  'Peter Walker',
  'Leslie Johnson',
  'Joe Fry',
  'Tony Rolt',
  'Alberto Ascari',
  'Raymond Sommer',
  'Luigi Villoresi',
  'José Froilán González',
  'Robert Manzon',
  'Maurice Trintignant',
  'Franco Rol',
  'Harry Schell',
  'Peter Whitehead',
  'Alfredo Pián',
  'Johnnie Parsons',
  'Bill Holland',
  'Mauri Rose',
  'Cecil Green',
  'Joie Chitwood',
  'Lee Wallard',
  'Walt Faulkner',
  'George Connor',
  'Paul Russo',
  'Pat Flaherty',
  'Myron Fohr',
  'Duane Carter',
  'Mack Hellings',
  'Jack McGrath',
  'Troy Ruttman',
  'Gene Hartley',
  'Jimmy Davies',
  'Johnny McDowell',
  'Walt Brown',
  'Travis Webb',
 